In [128]:
%load_ext autoreload
%autoreload 2

from src.base_classes.omic_data_loader import OmicDataLoader
from src.baseline_models_benchmarks.data_manager import CatOmicDataManager

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [129]:
mrna_loader = OmicDataLoader(
    data_dir="mds_data/splits_74/mrna",
)
mirna_loader = OmicDataLoader(
    data_dir="mds_data/splits_74/mirna_genes",
)
circrna_loader = OmicDataLoader(
    data_dir="mds_data/splits_74/circrna",
)
# pirna_loader = OmicDataLoader(
#     data_dir="mds_data/splits_74/pirna",
# )
te_loader = OmicDataLoader(
    data_dir="mds_data/splits_74/te_counts",
)

files
['train_0.csv', 'train_1.csv', 'train_2.csv', 'train_3.csv', 'train_4.csv']
['test_0.csv', 'test_1.csv', 'test_2.csv', 'test_3.csv', 'test_4.csv']
files
['train_0.csv', 'train_1.csv', 'train_2.csv', 'train_3.csv', 'train_4.csv']
['test_0.csv', 'test_1.csv', 'test_2.csv', 'test_3.csv', 'test_4.csv']
files
['train_0.csv', 'train_1.csv', 'train_2.csv', 'train_3.csv', 'train_4.csv']
['test_0.csv', 'test_1.csv', 'test_2.csv', 'test_3.csv', 'test_4.csv']
files
['train_0.csv', 'train_1.csv', 'train_2.csv', 'train_3.csv', 'train_4.csv']
['test_0.csv', 'test_1.csv', 'test_2.csv', 'test_3.csv', 'test_4.csv']


In [130]:
for fold_idx in range(5):
    train_df, test_df = mrna_loader.get_fold(fold_idx)

    print("fold: ", fold_idx)
    print(train_df["class"].value_counts(), test_df["class"].value_counts())

fold:  0
shape: (2, 2)
┌───────┬───────┐
│ class ┆ count │
│ ---   ┆ ---   │
│ i64   ┆ u32   │
╞═══════╪═══════╡
│ 0     ┆ 10    │
│ 1     ┆ 49    │
└───────┴───────┘ shape: (2, 2)
┌───────┬───────┐
│ class ┆ count │
│ ---   ┆ ---   │
│ i64   ┆ u32   │
╞═══════╪═══════╡
│ 0     ┆ 3     │
│ 1     ┆ 12    │
└───────┴───────┘
fold:  1
shape: (2, 2)
┌───────┬───────┐
│ class ┆ count │
│ ---   ┆ ---   │
│ i64   ┆ u32   │
╞═══════╪═══════╡
│ 0     ┆ 10    │
│ 1     ┆ 49    │
└───────┴───────┘ shape: (2, 2)
┌───────┬───────┐
│ class ┆ count │
│ ---   ┆ ---   │
│ i64   ┆ u32   │
╞═══════╪═══════╡
│ 0     ┆ 3     │
│ 1     ┆ 12    │
└───────┴───────┘
fold:  2
shape: (2, 2)
┌───────┬───────┐
│ class ┆ count │
│ ---   ┆ ---   │
│ i64   ┆ u32   │
╞═══════╪═══════╡
│ 0     ┆ 10    │
│ 1     ┆ 49    │
└───────┴───────┘ shape: (2, 2)
┌───────┬───────┐
│ class ┆ count │
│ ---   ┆ ---   │
│ i64   ┆ u32   │
╞═══════╪═══════╡
│ 0     ┆ 3     │
│ 1     ┆ 12    │
└───────┴───────┘
fold:  3
shape: (2, 2)
┌─

In [131]:
omic_data_loaders = {
    "mrna": mrna_loader,
    # "mirna": mirna_loader,
    # "circrna": circrna_loader,
    # "pirna": pirna_loader,
    # "te": te_loader,
}
odm = CatOmicDataManager(omic_data_loaders, n_splits=5)

In [132]:
from src.benchmarks.knn import KNNEvaluator

knn_eval = KNNEvaluator(
    data_manager=odm,
    n_trials=20,
    verbose=True,
    params={"k_lb": 1, "k_ub": 20},
)
_ = knn_eval.evaluate()
knn_eval.print_best_results()

[I 2024-11-10 15:04:39,192] A new study created in memory with name: no-name-68990823-a665-4ca6-b483-cc8dc229ac28


[I 2024-11-10 15:04:39,287] Trial 0 finished with value: 0.5751271113412196 and parameters: {'n_neighbors': 4}. Best is trial 0 with value: 0.5751271113412196.
[I 2024-11-10 15:04:39,354] Trial 1 finished with value: 0.5972672319727561 and parameters: {'n_neighbors': 3}. Best is trial 1 with value: 0.5972672319727561.
[I 2024-11-10 15:04:39,415] Trial 2 finished with value: 0.6977205806439 and parameters: {'n_neighbors': 20}. Best is trial 2 with value: 0.6977205806439.
[I 2024-11-10 15:04:39,476] Trial 3 finished with value: 0.7155098095140549 and parameters: {'n_neighbors': 11}. Best is trial 3 with value: 0.7155098095140549.


New best score: 0.575
Best model performance:
Accuracy: 0.865 ± 0.042
F1 Macro: 0.768 ± 0.089
F1 Weighted: 0.866 ± 0.041
New best score: 0.597
Best model performance:
Accuracy: 0.879 ± 0.025
F1 Macro: 0.777 ± 0.062
F1 Weighted: 0.875 ± 0.032
New best score: 0.698
Best model performance:
Accuracy: 0.919 ± 0.050
F1 Macro: 0.836 ± 0.111
F1 Weighted: 0.908 ± 0.062
New best score: 0.716
Best model performance:
Accuracy: 0.919 ± 0.050
F1 Macro: 0.852 ± 0.097
F1 Weighted: 0.913 ± 0.056


[I 2024-11-10 15:04:39,536] Trial 4 finished with value: 0.6804361055090542 and parameters: {'n_neighbors': 14}. Best is trial 3 with value: 0.7155098095140549.
[I 2024-11-10 15:04:39,596] Trial 5 finished with value: 0.7155098095140549 and parameters: {'n_neighbors': 18}. Best is trial 3 with value: 0.7155098095140549.
[I 2024-11-10 15:04:39,655] Trial 6 finished with value: 0.6977205806439 and parameters: {'n_neighbors': 20}. Best is trial 3 with value: 0.7155098095140549.
[I 2024-11-10 15:04:39,713] Trial 7 finished with value: 0.7155098095140549 and parameters: {'n_neighbors': 10}. Best is trial 3 with value: 0.7155098095140549.
[I 2024-11-10 15:04:39,802] Trial 8 finished with value: 0.6804361055090542 and parameters: {'n_neighbors': 14}. Best is trial 3 with value: 0.7155098095140549.
[I 2024-11-10 15:04:39,862] Trial 9 finished with value: 0.7155098095140549 and parameters: {'n_neighbors': 11}. Best is trial 3 with value: 0.7155098095140549.
[I 2024-11-10 15:04:39,924] Trial 10 

Best hyperparameters:
{'n_neighbors': 11}
Best model performance:
Accuracy: 0.919 ± 0.050
F1 Macro: 0.852 ± 0.097
F1 Weighted: 0.913 ± 0.056


In [133]:
from src.benchmarks.svm import SVMEvaluator

svm_eval = SVMEvaluator(
    data_manager=odm,
    n_trials=30,
    verbose=True,
    params={
        "C_lb": 0.01,
        "C_ub": 10,
        "rfe_step_range": (0.05, 0.2),
        "rfe_n_features_range": (100, 200),
    },
    mode="linear",
)
_ = svm_eval.evaluate()
svm_eval.print_best_results()

[I 2024-11-10 15:04:57,374] A new study created in memory with name: no-name-17b65a7d-695e-4e4c-9283-d9a803e51944


[I 2024-11-10 15:04:57,465] Trial 0 finished with value: 0.6497992012967035 and parameters: {'C': 0.5912776084859577, 'class_weight': None, 'rfe_step': 0.18135798798684793, 'rfe_n_features': 150}. Best is trial 0 with value: 0.6497992012967035.
[I 2024-11-10 15:04:57,574] Trial 1 finished with value: 0.6497992012967035 and parameters: {'C': 1.92761937700467, 'class_weight': 'balanced', 'rfe_step': 0.12740681956658, 'rfe_n_features': 111}. Best is trial 0 with value: 0.6497992012967035.
[I 2024-11-10 15:04:57,656] Trial 2 finished with value: 0.6477728936997709 and parameters: {'C': 0.04431683340011391, 'class_weight': 'balanced', 'rfe_step': 0.0561795144594292, 'rfe_n_features': 170}. Best is trial 0 with value: 0.6497992012967035.


New best score: 0.650
Best model performance:
Accuracy: 0.892 ± 0.053
F1 Macro: 0.814 ± 0.093
F1 Weighted: 0.895 ± 0.046


[I 2024-11-10 15:04:57,742] Trial 3 finished with value: 0.5947745825537253 and parameters: {'C': 6.239037597620005, 'class_weight': None, 'rfe_step': 0.16137130043179504, 'rfe_n_features': 150}. Best is trial 0 with value: 0.6497992012967035.
[I 2024-11-10 15:04:57,804] Trial 4 finished with value: 0.6497992012967035 and parameters: {'C': 1.8024991990621209, 'class_weight': None, 'rfe_step': 0.16636180097742892, 'rfe_n_features': 200}. Best is trial 0 with value: 0.6497992012967035.
[I 2024-11-10 15:04:57,914] Trial 5 finished with value: 0.5947745825537253 and parameters: {'C': 5.247154408538435, 'class_weight': None, 'rfe_step': 0.06264027199095756, 'rfe_n_features': 156}. Best is trial 0 with value: 0.6497992012967035.
[I 2024-11-10 15:04:58,008] Trial 6 finished with value: 0.5947745825537253 and parameters: {'C': 9.321484902226024, 'class_weight': None, 'rfe_step': 0.16378265009504533, 'rfe_n_features': 154}. Best is trial 0 with value: 0.6497992012967035.
[I 2024-11-10 15:04:58,

New best score: 0.679
Best model performance:
Accuracy: 0.906 ± 0.032
F1 Macro: 0.828 ± 0.068
F1 Weighted: 0.905 ± 0.031


[I 2024-11-10 15:04:58,755] Trial 14 finished with value: 0.6787024344579722 and parameters: {'C': 0.15592096415861068, 'class_weight': 'balanced', 'rfe_step': 0.0936740719697733, 'rfe_n_features': 129}. Best is trial 12 with value: 0.6787024344579722.
[I 2024-11-10 15:04:58,870] Trial 15 finished with value: 0.7331794285989254 and parameters: {'C': 0.10243547401908666, 'class_weight': 'balanced', 'rfe_step': 0.09295207471213159, 'rfe_n_features': 131}. Best is trial 15 with value: 0.7331794285989254.
[I 2024-11-10 15:04:58,960] Trial 16 finished with value: 0.6809510924096367 and parameters: {'C': 0.050080573100068776, 'class_weight': 'balanced', 'rfe_step': 0.1057608612630386, 'rfe_n_features': 120}. Best is trial 15 with value: 0.7331794285989254.
[I 2024-11-10 15:04:59,068] Trial 17 finished with value: 0.6422617001076846 and parameters: {'C': 0.07668997260540834, 'class_weight': 'balanced', 'rfe_step': 0.0832177373698388, 'rfe_n_features': 114}. Best is trial 15 with value: 0.7331

New best score: 0.733
Best model performance:
Accuracy: 0.918 ± 0.030
F1 Macro: 0.867 ± 0.041
F1 Weighted: 0.921 ± 0.024


[I 2024-11-10 15:04:59,147] Trial 18 finished with value: 0.6477728936997709 and parameters: {'C': 0.010268450364919926, 'class_weight': 'balanced', 'rfe_step': 0.11357879132217108, 'rfe_n_features': 139}. Best is trial 15 with value: 0.7331794285989254.
[I 2024-11-10 15:04:59,238] Trial 19 finished with value: 0.6809510924096367 and parameters: {'C': 0.0644825018841236, 'class_weight': 'balanced', 'rfe_step': 0.14146501839591458, 'rfe_n_features': 116}. Best is trial 15 with value: 0.7331794285989254.
[I 2024-11-10 15:04:59,340] Trial 20 finished with value: 0.728115464597245 and parameters: {'C': 0.10675039358903013, 'class_weight': 'balanced', 'rfe_step': 0.07651257359386716, 'rfe_n_features': 139}. Best is trial 15 with value: 0.7331794285989254.
[I 2024-11-10 15:04:59,445] Trial 21 finished with value: 0.728115464597245 and parameters: {'C': 0.1155640352564498, 'class_weight': 'balanced', 'rfe_step': 0.07635708884773021, 'rfe_n_features': 138}. Best is trial 15 with value: 0.73317

Best hyperparameters:
{'C': 0.10243547401908666, 'class_weight': 'balanced', 'rfe_step': 0.09295207471213159, 'rfe_n_features': 131}
Best model performance:
Accuracy: 0.918 ± 0.030
F1 Macro: 0.867 ± 0.041
F1 Weighted: 0.921 ± 0.024


Best model performance:
Accuracy: 0.938 ± 0.058
F1 Macro: 0.684 ± 0.258
F1 Weighted: 0.924 ± 0.064

In [134]:
from src.benchmarks.xgboost import XGBoostEvaluator

xgb_eval = XGBoostEvaluator(
    data_manager=odm,
    n_classes=2,
    n_trials=60,
    verbose=True,
)
_ = xgb_eval.evaluate()
xgb_eval.print_best_results()

[I 2024-11-10 15:05:07,559] A new study created in memory with name: no-name-55642bd2-7b22-48fc-accd-e23828aab96a


[I 2024-11-10 15:05:07,821] Trial 0 finished with value: 0.5844534524426352 and parameters: {'booster': 'dart', 'lambda': 7.492928775446501e-08, 'alpha': 5.381627270697887e-05, 'max_depth': 7, 'eta': 0.00027814311801822964, 'gamma': 5.622114885164392e-08, 'grow_policy': 'lossguide', 'sample_type': 'weighted', 'normalize_type': 'forest', 'rate_drop': 9.898766826438945e-06, 'skip_drop': 3.7159323985328213e-06}. Best is trial 0 with value: 0.5844534524426352.
[I 2024-11-10 15:05:07,966] Trial 1 finished with value: 0.601470599334144 and parameters: {'booster': 'dart', 'lambda': 1.552224504838912e-06, 'alpha': 2.2786942620206585e-07, 'max_depth': 6, 'eta': 0.13260049492450662, 'gamma': 6.8206524704820155e-06, 'grow_policy': 'depthwise', 'sample_type': 'weighted', 'normalize_type': 'forest', 'rate_drop': 1.2203233097980306e-05, 'skip_drop': 2.9255473827104788e-08}. Best is trial 1 with value: 0.601470599334144.


New best score: 0.584
Best model performance:
Accuracy: 0.879 ± 0.065
F1 Macro: 0.761 ± 0.170
F1 Weighted: 0.874 ± 0.070
New best score: 0.601
Best model performance:
Accuracy: 0.891 ± 0.069
F1 Macro: 0.765 ± 0.187
F1 Weighted: 0.882 ± 0.078


[I 2024-11-10 15:05:08,121] Trial 2 finished with value: 0.5046011800339922 and parameters: {'booster': 'dart', 'lambda': 0.016171049038895106, 'alpha': 0.002082197840640754, 'max_depth': 6, 'eta': 0.017179917538295533, 'gamma': 1.216646213963225e-06, 'grow_policy': 'depthwise', 'sample_type': 'weighted', 'normalize_type': 'forest', 'rate_drop': 6.571471924678859e-07, 'skip_drop': 5.284746709691009e-07}. Best is trial 1 with value: 0.601470599334144.
[I 2024-11-10 15:05:08,346] Trial 3 finished with value: 0.5587244304571011 and parameters: {'booster': 'gbtree', 'lambda': 0.03338751433843179, 'alpha': 0.0004583015929885411, 'max_depth': 3, 'eta': 3.6146554325305046e-07, 'gamma': 1.4213987892609452e-05, 'grow_policy': 'depthwise'}. Best is trial 1 with value: 0.601470599334144.
[I 2024-11-10 15:05:08,424] Trial 4 finished with value: 0.6000960923235792 and parameters: {'booster': 'gblinear', 'lambda': 8.442933396110046e-08, 'alpha': 0.003083949458276413}. Best is trial 1 with value: 0.6

New best score: 0.629
Best model performance:
Accuracy: 0.892 ± 0.032
F1 Macro: 0.795 ± 0.075
F1 Weighted: 0.887 ± 0.038


...

New best score: 0.663
Best model performance:
Accuracy: 0.906 ± 0.032
F1 Macro: 0.815 ± 0.062
F1 Weighted: 0.898 ± 0.038
New best score: 0.736
Best model performance:
Accuracy: 0.919 ± 0.050
F1 Macro: 0.868 ± 0.077
F1 Weighted: 0.922 ± 0.047


[I 2024-11-10 15:05:09,642] Trial 14 finished with value: 0.689882594756958 and parameters: {'booster': 'gblinear', 'lambda': 0.0012400409386503011, 'alpha': 9.823381843817801e-08}. Best is trial 13 with value: 0.7361252920039949.
[I 2024-11-10 15:05:09,739] Trial 15 finished with value: 0.689882594756958 and parameters: {'booster': 'gblinear', 'lambda': 0.0018350428712121975, 'alpha': 2.830542214304148e-07}. Best is trial 13 with value: 0.7361252920039949.
[I 2024-11-10 15:05:09,828] Trial 16 finished with value: 0.7711411011566484 and parameters: {'booster': 'gblinear', 'lambda': 1.1671508342458951e-05, 'alpha': 1.0034973626039293e-07}. Best is trial 16 with value: 0.7711411011566484.
[I 2024-11-10 15:05:09,986] Trial 17 finished with value: 0.5844534524426352 and parameters: {'booster': 'gbtree', 'lambda': 6.089704177346619e-06, 'alpha': 5.246215565102102e-06, 'max_depth': 9, 'eta': 0.00010091055188161564, 'gamma': 0.009275194216716336, 'grow_policy': 'lossguide'}. Best is trial 16 

New best score: 0.771
Best model performance:
Accuracy: 0.932 ± 0.042
F1 Macro: 0.886 ± 0.066
F1 Weighted: 0.933 ± 0.042


[I 2024-11-10 15:05:10,093] Trial 18 finished with value: 0.7361252920039949 and parameters: {'booster': 'gblinear', 'lambda': 4.696469162302939e-06, 'alpha': 4.7109607436227194e-07}. Best is trial 16 with value: 0.7711411011566484.
[I 2024-11-10 15:05:10,196] Trial 19 finished with value: 0.7012351321451076 and parameters: {'booster': 'gblinear', 'lambda': 2.0430996199947928e-05, 'alpha': 1.0627423496787941e-05}. Best is trial 16 with value: 0.7711411011566484.
[I 2024-11-10 15:05:10,384] Trial 20 finished with value: 0.6024938699160863 and parameters: {'booster': 'gbtree', 'lambda': 1.0543102835244368e-08, 'alpha': 5.994749276682222e-08, 'max_depth': 1, 'eta': 0.7326492278876501, 'gamma': 0.0009885616514817444, 'grow_policy': 'lossguide'}. Best is trial 16 with value: 0.7711411011566484.
[I 2024-11-10 15:05:10,489] Trial 21 finished with value: 0.7343191495811547 and parameters: {'booster': 'gblinear', 'lambda': 1.427761460634869e-06, 'alpha': 7.150038971379046e-07}. Best is trial 16

Best hyperparameters:
{'booster': 'gblinear', 'lambda': 1.1671508342458951e-05, 'alpha': 1.0034973626039293e-07}
Best model performance:
Accuracy: 0.932 ± 0.042
F1 Macro: 0.886 ± 0.066
F1 Weighted: 0.933 ± 0.042


In [92]:
knn_eval.save_results(results_file="logs/mds_disese_eval.csv", row_name="knn")
svm_eval.save_results(results_file="logs/mds_disese_eval.csv", row_name="svm")
xgb_eval.save_results(results_file="logs/mds_disese_eval.csv", row_name="xgb")

In [127]:
knn_eval.print_best_results()
svm_eval.print_best_results()
xgb_eval.print_best_results()

Best model performance:
Accuracy: 0.923 ± 0.084
F1 Macro: 0.825 ± 0.208
F1 Weighted: 0.933 ± 0.072
Best model performance:
Accuracy: 0.954 ± 0.038
F1 Macro: 0.688 ± 0.255
F1 Weighted: 0.932 ± 0.056
Best model performance:
Accuracy: 0.954 ± 0.038
F1 Macro: 0.688 ± 0.255
F1 Weighted: 0.932 ± 0.056


In [70]:
print(odm.train_y)
print(odm.test_y)

None
None


In [71]:
odm.get_split(1)

(shape: (53, 400)
 ┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
 │ ENSG00000 ┆ ENSG00000 ┆ ENSG00000 ┆ ENSG00000 ┆ … ┆ hsa-miR-2 ┆ hsa-miR-6 ┆ hsa-miR-6 ┆ hsa-miR- │
 │ 113552    ┆ 204252    ┆ 171067    ┆ 078804    ┆   ┆ 9b-3p     ┆ 808-3p    ┆ 60-3p     ┆ 337-3p   │
 │ ---       ┆ ---       ┆ ---       ┆ ---       ┆   ┆ ---       ┆ ---       ┆ ---       ┆ ---      │
 │ f64       ┆ f64       ┆ f64       ┆ f64       ┆   ┆ f64       ┆ f64       ┆ f64       ┆ f64      │
 ╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
 │ 0.101906  ┆ 0.67432   ┆ 0.553844  ┆ 0.427465  ┆ … ┆ 0.332964  ┆ 0.491693  ┆ 0.228302  ┆ 0.478208 │
 │ 0.211153  ┆ 0.79269   ┆ 0.443358  ┆ 0.538522  ┆ … ┆ 0.231004  ┆ 0.367655  ┆ 0.0       ┆ 0.463624 │
 │ 0.248448  ┆ 0.901163  ┆ 0.376432  ┆ 0.818353  ┆ … ┆ 0.567736  ┆ 0.200131  ┆ 0.0       ┆ 0.377795 │
 │ 0.0       ┆ 0.855781  ┆ 0.095358  ┆ 0.86714   ┆ … ┆ 0.390975 